# Intro



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import data from xlsx

In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/deep_learning_training/experiments/titanic/titanic.xlsx', sheet_name='all_rows')
df = df.dropna(subset=['Age', 'Embarked'])
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
889,1,1,male,26.0,0,0,30.0000,C


## Normalize / adapt data

In [3]:
import torch
import numpy as np
expected_output = torch.tensor(np.array(df.Survived))
expected_output.shape

torch.Size([712])

In [113]:
input_dict = []

def normalize(x):
  return (x - x.min()) / (x.max() - x.min())

input_dict.append(np.where(np.array(df.Pclass) == 1, 1, 0)) # First class
input_dict.append(np.where(np.array(df.Pclass) == 2, 1, 0)) # Second class
input_dict.append(np.where(np.array(df.Sex) == 'male', 1, 0)) # Gender
input_dict.append(normalize(np.array(df.Age))) # Age
input_dict.append(normalize(1 + np.array(df.SibSp) + np.array(df.Parch))) # Family
input_dict.append(normalize(np.log10(np.array(df.Fare + 1)))) # Fare
input_dict.append(np.where(np.array(df.Embarked) == 'S',1,0)) # Embarked S
input_dict.append(np.where(np.array(df.Embarked) == 'C',1,0)) # Embarked C
input_dict.append(np.ones(len(df.Age))) # Ones


## Run linear gradient descent

In [140]:
torch.set_printoptions(precision=2, sci_mode=False)

input_matrix = torch.tensor(input_dict)

train_set_size = 600
train_set = input_matrix[:, :train_set_size]
validation_set = input_matrix[:, train_set_size:]

expected_output_train_set = expected_output[:train_set_size]
expected_output_validation_set = expected_output[train_set_size:]



In [233]:
np.random.seed(42)
input_weights = torch.tensor(np.random.rand(train_set.shape[0]) - .5)
input_weights.requires_grad_()

learning_rate = .1

def forward(set, weights):
  layer1 = torch.mv(set.T, weights)
  return torch.relu(layer1)

def mse(pred, act):
  return torch.mean(torch.pow(torch.sub(pred,act), 2))

for epoch in range(64):
  generated_output = forward(train_set, input_weights)
  loss = mse(expected_output_train_set, generated_output)
  loss.backward()
  with torch.no_grad():
    input_weights -= input_weights.grad * learning_rate
    input_weights.grad.zero_()

generated_output = forward(validation_set, input_weights)
correct = (expected_output_validation_set == torch.round(generated_output)).sum() / len(generated_output)
print(f'Validation set correct: {correct * 100}%')


Validation set correct: 81.25%


## Run 2 layers neural net

In [252]:
np.random.seed(42)
input_weights1 = torch.tensor(np.random.rand(2, train_set.shape[0]) - .5)
input_weights1.requires_grad_()

input_weights2 = torch.tensor(np.random.rand(1, len(input_weights1)) - .5)
input_weights2.requires_grad_()

learning_rate = .1

def forward(set, weights1, weights2):
  layer1 = torch.matmul(weights1, set)
  layer2 = torch.matmul(weights2, layer1)
  return layer2.squeeze(0)

def mse(pred, act):
  return torch.mean(torch.pow(torch.sub(pred,act), 2))

for epoch in range(64):
  generated_output = forward(train_set, input_weights1, input_weights2)
  loss = mse(expected_output_train_set, generated_output)
  loss.backward()
  with torch.no_grad():
    input_weights1 -= input_weights1.grad * learning_rate
    input_weights2 -= input_weights2.grad * learning_rate
    input_weights1.grad.zero_()
    input_weights2.grad.zero_()

  # print(f'Loss at epoch {epoch}: {loss:.3f}')

generated_output = forward(validation_set, input_weights1, input_weights2)
correct = (expected_output_validation_set == torch.round(generated_output)).sum() / len(generated_output)
print(f'Validation set correct: {correct * 100}%')


Validation set correct: 83.03571319580078%


## Use pytorch (?) to optimize matrix multiplication